In [104]:
import snap
from snap import TUNGraph
import os
import sys
import operator
import pandas as pd
import subprocess

# Global variables
data_dir = "../data"
N = 89577277
L = 434193958

# 1. Build MMR Graph
Calculating the network metrics doesn't necessarily require including edge metadata in the graph, therefore **SNAP.Py** comes handy in this case. Let's import the MMR data in an **undirected graph** data structure and build the main network.

Since SNAP.Py allows inserting vertices with a **node ID**, I don't need to read the `usernames.csv` file, since all IDs from 0 to $(N-1)$ are existing for sure, where $N=89.577.277$. By the other hand, I may read the edges from the `mmr_encoded_final.csv` file and ignore the edge metadata column including the periods array.

In [57]:
def read_large_file(file_object):
    while True:
        data = file_object.readline()
        if not data:
            break
        yield data.rstrip('\n')

# We ignore prop for the moment
def process_edge_line(line):
    source, target, prop = line.split(',')
    return int(source), int(target)

def build_mmr_graph():
    # Create main graph object
    graph = TUNGraph.New()

    # First add vertices
    print "Adding vertices..."
    for i in xrange(0, N):
        graph.AddNode(i)
    print "Done adding vertices, now adding links..."

    # Add links by reading them from MMR data file
    with open(os.path.join(data_dir,"mmr_encoded_final.csv")) as io:
        counter = 0
        for line in read_large_file(io):
            source, target = process_edge_line(line)
            graph.AddEdge(source, target)
            counter += 1
            if counter % 50000000 == 0:
                print "Added %d links..." %counter
        print "Done! Total links: %d" %counter
        assert counter == L
    return graph

In [58]:
%%time
graph = build_mmr_graph()

Adding vertices...
Done adding vertices, now adding links...
Added 50000000 links...
Added 100000000 links...
Added 150000000 links...
Added 200000000 links...
Added 250000000 links...
Added 300000000 links...
Added 350000000 links...
Added 400000000 links...
Done! Total links: 434193958


In [65]:
%%bash
free -h

              total        used        free      shared  buff/cache   available
Mem:           125G         16G         44G         86M         65G        108G
Swap:           63G        256M         63G


The whole graph loaded in memory takes *less than 16GB* of RAM, which is significantly uncomparable with NetworkX's performance.

# 2. Degree Distribution and Node Degrees
SNAP.Py comes with a built-in function to calculate the histogram values needed to plot the degree distribution. In order to be able to plot the distribution afterwards (and to avoid re-calculating it) we may save these values to a file, so that they can be conveniently loaded from another notebook. Values are saved in CSV format, such that the first column indicates the $x$-axis value ($k$ = degree), and the second column the $y$-axis value (number of $k$-degree nodes).

The degree distribution represents a core feature for network analysis, because it explains what is the probability $p_k$ that, given a random node, it has degree $k$. Therefore one may calculate $p_k$ as: $p_k=\frac{N_k}{N}$, where $N_k$ is the number of degree-$k$ nodes. Furthermore, $p_k$ determines many network phenomena such as network robustness.

In [69]:
%%time
degree_distribution_filename = "mmr_degree_distribution.csv"

def calculate_degree_distribution(graph, out_filename):
    DegToCntV = snap.TIntPrV()
    snap.GetDegCnt(graph, DegToCntV)
    with open(os.path.join(data_dir, degree_distribution_filename), "w") as out:
        for item in DegToCntV:
            # Probability has to be normalized
            p = item.GetVal2() * 1.0 / N
            out.write(",".join([str(item.GetVal1()),str(p)]) + "\n")
    return DegToCntV

DegToCntV = calculate_degree_distribution(graph, degree_distribution_filename)

CPU times: user 1.99 s, sys: 0 ns, total: 1.99 s
Wall time: 1.99 s


Calculating the degree distribution with SNAP.Py highlights how much it's extremely efficient and performant even with large-scale networks.

## 2.1 Usernames with highest $k$

In [76]:
n_id_max_deg = snap.GetMxDegNId(graph)
print "The node with the highest degree has ID %d" %n_id_max_deg

The node with the highest degree has ID 54790022


In [77]:
%%bash
../scripts/get_username.sh 54790022

cocanomc,54790022


In [83]:
InDegV = snap.TIntPrV()
snap.GetNodeInDegV(graph, InDegV)
node_degrees = [(item.GetVal1(), item.GetVal2()) for item in InDegV]

# Sort list by degree
node_degrees = sorted(node_degrees, key=operator.itemgetter(1))

Let's show who are the top 20 nodes with the highest degree and try to figure out if any of them correspond to any well known Twitter username:

In [105]:
data = sorted(node_degrees[-20:], key=operator.itemgetter(1), reverse=True)
node_ids = [el[0] for el in data]
usernames = []
for i in node_ids:
    p = subprocess.Popen(['../scripts/get_username.sh', str(i)], stdout=subprocess.PIPE)
    output = p.communicate()[0]
    usernames.append(output)

In [108]:
# Cleanup usernames
usernames = [el.split(',')[0] for el in usernames]

# Prepare dataframe data
data = [[el[0], el[1][0], el[1][1]] for el in zip(usernames, data)]

In [118]:
df = pd.DataFrame(data, columns=["Username", "Node ID (Encoding)", "Degree"])
df

,Username,Node ID (Encoding),Degree
0,cocanomc,54790022,41359
1,telkomsel,61888,33341
2,stccare,272020,31578
3,virginmedia,113519,30738
4,xboxsupport,112622,27853
5,americanair,38831,25676
6,atviassist,13925,23270
7,tesco,34364,20606
8,btcare,46286,19399
9,indosatcare,64396,18911
